# Clustering
## K-means
## Gaussian Mixture Models
## Spectral
## Hierarchical
### Agglomerative
### Divisive
## DBSCAN

# Matrix Factorization
## Singular Value Decomposition
## Principal Component Analysis
## Independent Component Analysis
## Non-Negative Matrix Factorization
## Latent Dirichlet Allocation\

# Manifold Learning
## t-SNE
## UMAP
## Factor Analysis

# Anomaly Detection
## Distance-Based Methods
## Density-Based Methods

# Association Rule Learning
## Apriori Algorithm
## Eclat Algorithm
## FP-Growth Algorithm